In [1]:
from thop import profile
import torch
import torch.nn as nn
from opts import parser
import os, sys, argparse
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('..'))))
from manjin_models import TSN
from resnet_TSM import matching_layer

In [2]:
mainFolder="net_runs"
subFolder="TSM_matchflow_consecutive_segment_resnet18_something_run16"
snap_pref="TSM_resnet"

train_path="data/something_train.txt"
val_path="data/something_val.txt"

In [3]:
dataset_name="something"
netType1="TSM"
netType2="TSM_resnet50"
netType3="TSM_flow"
netType4="TSM_flow_resnet50"
netType5="I3D"
netType6="I3D_flow"
batch_size=1
learning_rate=0.01
num_segments_8=8
num_segments_16=16
num_segments_32=32
num_segments_128=128
mode=1
dropout=0.0
iter_size=1
num_workers=5

In [4]:
input1 = torch.rand(num_segments_8,3,224,224).cuda()
input2 = torch.rand(num_segments_16,3,224,224).cuda()
input3 = torch.rand(num_segments_32,3,224,224).cuda()
input4 = torch.rand(num_segments_128,3,224,224).cuda()

In [5]:
sys.argv = ['main.py', dataset_name, 'RGB', train_path, val_path, '--arch',
            str(netType1), '--num_segments', str(num_segments_8), '--mode', str(mode),
            '--gd', '200', '--lr', str(learning_rate), '--lr_steps',
            '20', '30', '--epochs', '35', '-b', str(batch_size), '-i',
            str(iter_size), '-j', str(num_workers), '--dropout',
            str(dropout),
            '--consensus_type', 'avg', '--eval-freq', '1', '--rgb_prefix', 'img_',
            '--pretrained_parts', 'finetune', '--no_partialbn',
            '-p', '20', '--nesterov', 'True']

In [6]:
args = parser.parse_args()

In [7]:
args_dict = args.__dict__
print("------------------------------------")
print(args.arch+" Configurations:")
for key in args_dict.keys():
    print("- {}: {}".format(key, args_dict[key]))
print("------------------------------------")

------------------------------------
TSM Configurations:
- dataset: something
- modality: RGB
- train_list: data/something_train.txt
- val_list: data/something_val.txt
- arch: TSM
- num_segments: 8
- mode: 1
- consensus_type: avg
- pretrained_parts: finetune
- k: 3
- dropout: 0.0
- loss_type: nll
- rep_flow: False
- epochs: 35
- batch_size: 1
- iter_size: 1
- lr: 0.01
- lr_steps: [20.0, 30.0]
- momentum: 0.9
- weight_decay: 0.0005
- clip_gradient: 200.0
- no_partialbn: True
- nesterov: True
- num_long_cycles: 0
- num_short_cycles: 0
- last_cycle_tune: False
- print_freq: 20
- eval_freq: 1
- workers: 5
- resume: 
- evaluate: False
- snapshot_pref: 
- val_output_folder: 
- start_epoch: 0
- gpus: None
- flow_prefix: img_
- rgb_prefix: img_
------------------------------------


In [8]:
if args.dataset == 'ucf101':
    num_class = 101
    rgb_read_format = "{:05d}.jpg"
elif args.dataset == 'hmdb51':
    num_class = 51
    rgb_read_format = "{:05d}.jpg"        
elif args.dataset == 'kinetics':
    num_class = 400
    rgb_read_format = "{:05d}.jpg"
elif args.dataset == 'something':
    num_class = 174
    rgb_read_format = "{:05d}.jpg"
elif args.dataset == 'tinykinetics':
    num_class = 150
    rgb_read_format = "{:05d}.jpg"        
else:
    raise ValueError('Unknown dataset '+args.dataset)

In [9]:
model1 = TSN(num_class, args.num_segments, args.pretrained_parts, args.modality,
                base_model=netType1,
                consensus_type=args.consensus_type, dropout=args.dropout, partial_bn=not args.no_partialbn).cuda()
# model2 = TSN(num_class, args.num_segments, args.pretrained_parts, args.modality,
#                 base_model=netType2,
#                 consensus_type=args.consensus_type, dropout=args.dropout, partial_bn=not args.no_partialbn).cuda()
model3 = TSN(num_class, args.num_segments, args.pretrained_parts, args.modality,
                base_model=netType3,
                consensus_type=args.consensus_type, dropout=args.dropout, partial_bn=not args.no_partialbn).cuda()
# model4 = TSN(num_class, args.num_segments, args.pretrained_parts, args.modality,
#                 base_model=netType4,
#                 consensus_type=args.consensus_type, dropout=args.dropout, partial_bn=not args.no_partialbn).cuda()
# model9 = TSN(num_class, args.num_segments, args.pretrained_parts, args.modality,
#                 base_model=netType5,
#                 consensus_type=args.consensus_type, dropout=args.dropout, partial_bn=not args.no_partialbn).cuda()
# model10 = TSN(num_class, args.num_segments, args.pretrained_parts, args.modality,
#                 base_model=netType6,
#                 consensus_type=args.consensus_type, dropout=args.dropout, partial_bn=not args.no_partialbn).cuda()


Initializing TSN with base model: TSM.
TSN Configurations:
    input_modality:     RGB
    num_segments:       8
    new_length:         1
    consensus_module:   avg
    dropout_ratio:      0.0
    representation flow:      False
        
conv1.weight layer has pretrained weights
bn1.running_mean layer has pretrained weights
bn1.running_var layer has pretrained weights
bn1.weight layer has pretrained weights
bn1.bias layer has pretrained weights
layer1.0.conv1.weight layer has pretrained weights
layer1.0.bn1.running_mean layer has pretrained weights
layer1.0.bn1.running_var layer has pretrained weights
layer1.0.bn1.weight layer has pretrained weights
layer1.0.bn1.bias layer has pretrained weights
layer1.0.conv2.weight layer has pretrained weights
layer1.0.bn2.running_mean layer has pretrained weights
layer1.0.bn2.running_var layer has pretrained weights
layer1.0.bn2.weight layer has pretrained weights
layer1.0.bn2.bias layer has pretrained weights
layer1.1.conv1.weight layer has pret

In [10]:
args.num_segments=num_segments_16

In [11]:
# model5 = TSN(num_class, args.num_segments, args.pretrained_parts, args.modality,
#                 base_model=netType1,
#                 consensus_type=args.consensus_type, dropout=args.dropout, partial_bn=not args.no_partialbn).cuda()
# model6 = TSN(num_class, args.num_segments, args.pretrained_parts, args.modality,
#                 base_model=netType2,
#                 consensus_type=args.consensus_type, dropout=args.dropout, partial_bn=not args.no_partialbn).cuda()
# model7 = TSN(num_class, args.num_segments, args.pretrained_parts, args.modality,
#                 base_model=netType3,
#                 consensus_type=args.consensus_type, dropout=args.dropout, partial_bn=not args.no_partialbn).cuda()
# model8 = TSN(num_class, args.num_segments, args.pretrained_parts, args.modality,
#                 base_model=netType4,
#                 consensus_type=args.consensus_type, dropout=args.dropout, partial_bn=not args.no_partialbn).cuda()

In [12]:
args.num_segments=num_segments_32

In [13]:
# model9 = TSN(num_class, args.num_segments, args.pretrained_parts, args.modality,
#                 base_model=netType2,
#                 consensus_type=args.consensus_type, dropout=args.dropout, partial_bn=not args.no_partialbn).cuda()

In [14]:
args.num_segments=num_segments_128

In [15]:
# model10 = TSN(num_class, args.num_segments, args.pretrained_parts, args.modality,
#                 base_model=netType2,
#                 consensus_type=args.consensus_type, dropout=args.dropout, partial_bn=not args.no_partialbn).cuda()

In [16]:
# temperature
increase = pow(1.05, 30)
temperature = 10 * increase

In [17]:
flops1, params1 = profile(model1, inputs=(input1, temperature, 1), verbose=False)
# flops2, params2 = profile(model2, inputs=(input1, temperature, ), verbose=False)
flops3, params3 = profile(model3, inputs=(input1, temperature, 1), verbose=True)
# flops4, params4 = profile(model4, inputs=(input1, temperature, ), verbose=False)
# flops5, params5 = profile(model5, inputs=(input2, temperature, ), verbose=False)
# flops6, params6 = profile(model6, inputs=(input2, temperature, ), verbose=False)
# flops7, params7 = profile(model7, inputs=(input2, temperature, ), verbose=False)
# flops8, params8 = profile(model8, inputs=(input2, temperature, ), verbose=False)

No BN layer Freezing.
No BN layer Freezing.
No BN layer Freezing.
Register FLOP counter for module Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
Register FLOP counter for module BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Register FLOP counter for module BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Register FLOP counter for module ReLU(inplace)
Register FLOP counter for module Sigmoid()
Register FLOP counter for module MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
Register FLOP counter for module Softmax()
Register FLOP counter for module ReLU()
Register FLOP counter for module SpatialCorrelationSampler()
Register FLOP counter for module Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
Register FLOP counter for module BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Register FLOP counter for module

In [18]:
# flops9, params9 = profile(model9, inputs=(input1, temperature, ), verbose=False)
# flops10, params10 = profile(model10, inputs=(input1, temperature, ), verbose=False)

In [19]:
def human_format(num):
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    # add more suffixes if you need them
    return '%.4f%s' % (num, ['', 'K', 'M', 'G', 'T', 'P'][magnitude])

In [20]:
print('%s,%d\t\t\tFLOPs: %s, \tParams : %s' % (netType1, num_segments_8, human_format(flops1), human_format(params1)))
# print('%s,%d\t\tFLOPs: %s, \tParams : %s' % (netType2, num_segments_8, human_format(flops2), human_format(params2)))
print('%s,%d\t\tFLOPs: %s, \tParams : %s' % (netType3, num_segments_8, human_format(flops3), human_format(params3)))
# print('%s,%d\tFLOPs: %s, \tParams : %s' % (netType4, num_segments_8, human_format(flops4), human_format(params4)))
# print('='*70)
# print('%s,%d\t\t\tFLOPs: %s, \tParams : %s' % (netType1, num_segments_16, human_format(flops5), human_format(params5)))
# print('%s,%d\t\tFLOPs: %s, \tParams : %s' % (netType2, num_segments_16, human_format(flops6), human_format(params6)))
# print('%s,%d\t\tFLOPs: %s, \tParams : %s' % (netType3, num_segments_16, human_format(flops7), human_format(params7)))
# print('%s,%d\tFLOPs: %s, \tParams : %s' % (netType4, num_segments_16, human_format(flops8), human_format(params8)))

TSM,8			FLOPs: 14.5896G, 	Params : 11.2660M
TSM_flow,8		FLOPs: 14.9433G, 	Params : 11.2961M


In [21]:
# print('%s,%d\t\tFLOPs: %s, \tParams : %s' % (netType2, num_segments_8, human_format(flops2), human_format(params2)))
# print('%s,%d\t\tFLOPs: %s, \tParams : %s' % (netType2, num_segments_16, human_format(flops6), human_format(params6)))
# print('%s,%d\t\tFLOPs: %s, \tParams : %s' % (netType2, num_segments_32, human_format(flops9), human_format(params9)))
# print('%s,%d\t\tFLOPs: %s, \tParams : %s' % (netType2, num_segments_128, human_format(flops10), human_format(params10)))

In [22]:
# print('%s,%d\t\tFLOPs: %s, \tParams : %s' % (netType5, num_segments_8, human_format(flops9), human_format(params9)))
# print('%s,%d\t\tFLOPs: %s, \tParams : %s' % (netType6, num_segments_8, human_format(flops10), human_format(params10)))